In [ ]:
#1. Завантажуємо датасет для роботи
# Встановлюємо keggle
! pip install kaggle

In [ ]:
# Монтуємо Google Drive
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Робимо папку та копіюємо в неї файл з API key
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/kaggle.json
# Обмежуємо доступ до файлу
! chmod 600 ~/.kaggle/kaggle.json
# Завантажуємо датасет
! kaggle datasets download alessiocorrado99/animals10
# Розпаковуємо архів
! unzip animals10.zip

In [ ]:
#2. Розділяємо дані на навчальні та тестові
from tensorflow.keras.preprocessing.image import ImageDataGenerator
img_size = (227, 227)
batch_size = 32
extract_path="/content/raw-img"
datagen = ImageDataGenerator(
    rescale=1.0/255, # Нормалізація
    validation_split=0.2 # 20% даних під валідацію
)
train_generator = datagen.flow_from_directory(
    extract_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)
val_generator = datagen.flow_from_directory(
    extract_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

#3. Створюємо шари нейромережі
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
model = Sequential([
    Input(shape=(227, 227, 3)),
    Conv2D(96, (11, 11), strides=4, activation='relu'),
    BatchNormalization(),
    MaxPooling2D((3, 3), strides=2),
    Conv2D(256, (5, 5), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((3, 3), strides=2),
    Conv2D(384, (3, 3), activation='relu', padding='same'),
    Conv2D(384, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((3, 3), strides=2),
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax') # 10 класів
])

from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=[ 'accuracy'])
print(model.summary())

#4. Навчаємо модель
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator
)

In [ ]:
#5. Виводимо точність та втрати
loss, accuracy = model.evaluate(val_generator)
print(f"Точність на валідації: {accuracy:.4f}")
print(f"Втрати (Loss): {loss:.4f}")

#6. Будуємо графіки точності та втрати
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Точність на тренуванні')
plt.plot(epochs_range, val_acc, label='Точність на валідації')
plt.legend()
plt.title('Графік точності')
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Втрати на тренуванні')
plt.plot(epochs_range, val_loss, label='Втрати на валідації')
plt.legend()
plt.title('Графік втрат')
plt.show()

#7. Перевіряємо розпізнавання на випадковому зображенні
import numpy as np
import random
import os
from tensorflow.keras.preprocessing import image
class_names = list(train_generator.class_indices.keys())
random_class = random.choice(class_names)
random_image_path = random.choice(os.listdir(f"{extract_path}/{random_class}"))
img_path = f"{extract_path}/{random_class}/{random_image_path}"
img = image.load_img(img_path, target_size=(227, 227))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)
predictions = model.predict(img_array)
print(predictions)
predicted_class = class_names[np.argmax(predictions)]
plt.imshow(img)
plt.axis('off') 
plt.title(f"Очікуваний: {random_class}\nПередбачений: {predicted_class}")
plt.show()


import os
import random
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
num_images = 1024
batch_size = 128
output_csv = "classification_results.csv"
all_images = []
for class_name in os.listdir(extract_path):
    class_dir = os.path.join(extract_path, class_name)
    if os.path.isdir(class_dir):
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            all_images.append((img_path, class_name))
selected_images = random.sample(all_images, num_images)
def load_batch(image_data):
    images = []
    paths = []
    true_classes = []
    for img_path, true_class in image_data:
        img = image.load_img(img_path, target_size=(227, 227))
        img_array = image.img_to_array(img) / 255.0
        images.append(img_array)
        paths.append(img_path)
        true_classes.append(true_class)
    return np.array(images), paths, true_classes

results = []
for i in range(0, num_images, batch_size):
    batch_data = selected_images[i:i+batch_size]
    batch_images, batch_paths, batch_true_classes = load_batch(batch_data)
    predictions = model.predict(batch_images)
    predicted_classes = np.argmax(predictions, axis=1)
    for j in range(len(batch_paths)):
        results.append([batch_paths[j], batch_true_classes[j], class_names[pr edicted_classes[j]]])

df = pd.DataFrame(results, columns=["Шлях до файлу", "Справжній клас", "Розпі знаний клас"])
df.to_csv(output_csv, index=False, encoding="utf-8")
print(f"Результати збережені у {output_csv}")